## Dash Cytoscape

Dash Cytoscape - это компонент модуль для создания легко настраиваемых, высокопроизводительных, интерактивных и веб-визуализаций сетей. Он использует Cytoscape.js, хорошо интегрирован с макетами Dash и обратными вызовами.

Элемент (узел или связь) описывается словарем. Ключи:
* `data` - характеристики элемента (ID, Label, ...)
    * для ребер `data` должен содержать ключи `source` и `target`
* `position` - координаты (только для узлов)
* булевы атрибуты:
    * `locked` - True, если позицию узла нельзя менять
    * `selected` - True, если элемент выбран сразу после инициализации
    * `selectable` - True, если элемент может быть выбран
    * `grabbable` -  True, сли узел может быть захвачен и перемещен пользователем
* `classes` - класс или классы элемента (через пробел)

Подобно классам CSS, классы элементов используются для стилизации групп элементов с помощью селектора. Элементу можно присвоить  класс или несколько классов (разделенных пробелом).

In [22]:
%%file 06_dash_cytoscape/path.py
from dash import Dash, html, dcc
import dash_cytoscape as cyto
import networkx as nx
G = nx.karate_club_graph()
pos = nx.spring_layout(G, seed=42)

my_stylesheet = [
    # Стиль для узлов
    {
        'selector': 'node',
        'style': {
            'content': 'data(label)'
        }
    },
    # Стиль для классов
    {
        'selector': '.red',
        'style': {
            'background-color': 'red',
            'line-color': 'red'
        }
    },
    {
        'selector': '.triangle',
        'style': {
            'shape': 'triangle'
        }
    }
]


app = Dash(__name__)

app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape-two-nodes',
        layout={'name': 'preset'}, # располагает в соответствии с координатами узлов
        style={'width': '100%', 'height': '800px'},
        stylesheet=my_stylesheet, # описание всех используемых классов
        elements=[
            {"data": {"id": 0, "label": "0 (locked)"}, "position": {"x": 0, "y": 0}, "locked": True, "classes": "red"},
            {"data": {"id": 1, "label": "1 (selected)"}, "position": {"x": 150, "y": 0}, "selected": True, "classes": "triangle"},
            {"data": {"id": 2, "label": "2 (!selectable)"}, "position": {"x": 250, "y": 0}, "selectable": False, "classes": "triangle"},
            {"data": {"id": 3, "label": "3 (!grabbable)"}, "position": {"x": 350, "y": 0}, "grabbable": False, "classes": "red"},
            {"data": {"source": 0, "target": 1}},
            {"data": {"source": 1, "target": 2}},
            {"data": {"source": 2, "target": 3}},
        ]
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting 06_dash_cytoscape/path.py


![Cyto Path](06_dash_cytoscape/cyto_path.png)

## Составные узлы

Составные узлы - это узлы, которые содержат (родительские) или содержатся (дочерние) внутри другого узла. Родительский узел не имеет ни позиции, ни размера, поскольку эти значения автоматически вычисляются на основе того, как настроены дочерние узлы.

In [21]:
%%file 06_dash_cytoscape/compound.py
from dash import Dash, html, dcc
import dash_cytoscape as cyto
import networkx as nx
G = nx.karate_club_graph()
pos = nx.spring_layout(G, seed=42)

app = Dash(__name__)

app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape-two-nodes',
        layout={'name': 'preset'}, # располагает в соответствии с координатами узлов
        style={'width': '100%', 'height': '800px'},
        stylesheet=[ # описание всех используемых классов
            {
                'selector': 'node',
                'style': {
                    'content': 'data(label)'
                },
            },
            {
                'selector': '.countries',
                'style': {
                    'width': 5
                }
            },
            {
                'selector': '.cities',
                'style': {
                    'line-style': 'dashed'
                }
            },
        ], 
        elements=[
            # родительские узлы
            {"data": {"id": "rus", "label": "Россия"}},
            {"data": {"id": "br", "label": "Беларусь"}},
            # дочерние узлы
            {"data": {"id": "mos", "label": "Москва", "parent": "rus"}, "position": {"x": 100, "y": 100}},
            {"data": {"id": "dg", "label": "Долгопрудный", "parent": "rus"}, "position": {"x": 100, "y": 200}},
            {"data": {"id": "min", "label": "Минск", "parent": "br"}, "position": {"x": 400, "y": 200}},
            # связи
            {"data": {"source": "rus", "target": "br"}, "classes": "countries"},
            {"data": {"source": "dg", "target": "mos"}, "classes": "cities"},
            {"data": {"source": "min", "target": "mos"}, "classes": "cities"},
            
        ]
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting 06_dash_cytoscape/compound.py


![Cyto Compound](06_dash_cytoscape/cyto_compound.png)

## Макеты

Макет определяет, как узлы будут расположены на экране. При создании трейса `cyto.Cytoscape` нужно указать аргумент `layout` в виде словаря, в котором обязательно должен быть ключ `name`. Значение `name` может быть следующим ([документация](https://js.cytoscape.org/#layouts)):
* preset - расположение на основе заданных координат
* random - в произвольных положениях в окне просмотра
* grid - в виде сетки
* circle - в виде окружности
* concentric - в виде концентрических окружностей
* breathfirst - в виде иерархии (подход для деревьев)
* cose - на основе физ. симуляции.

Каждый из этих макетов имеет дополнительные аргументы, которые влияют на работу алгоритма (кол-во столбцов и строк для решетки, откуда начинать выполнять BFS и т.д.)


In [32]:
%%file 06_dash_cytoscape/layout.py
from dash import Dash, html, dcc
import dash_cytoscape as cyto
import networkx as nx
import math

def city_graph():
    nodes = [
        {
            'data': {'id': short, 'label': label},
            'position': {'x': 20 * lat, 'y': -20 * long}
        }
        for short, label, long, lat in (
            ('la', 'Los Angeles', 34.03, -118.25),
            ('nyc', 'New York', 40.71, -74),
            ('to', 'Toronto', 43.65, -79.38),
            ('mtl', 'Montreal', 45.50, -73.57),
            ('van', 'Vancouver', 49.28, -123.12),
            ('chi', 'Chicago', 41.88, -87.63),
            ('bos', 'Boston', 42.36, -71.06),
            ('hou', 'Houston', 29.76, -95.37)
        )
    ]

    edges = [
        {'data': {'source': source, 'target': target}}
        for source, target in (
            ('van', 'la'),
            ('la', 'chi'),
            ('hou', 'chi'),
            ('to', 'mtl'),
            ('mtl', 'bos'),
            ('nyc', 'bos'),
            ('to', 'hou'),
            ('to', 'nyc'),
            ('la', 'nyc'),
            ('nyc', 'bos')
        )
    ]
    elements = nodes + edges
    return elements


app = Dash(
    __name__,
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
)
elements = city_graph()

app.layout = html.Div([
        html.Div(
            [
                html.Div([
                    'cytoscape-grid',
                    cyto.Cytoscape(
                        id='cytoscape-grid',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'grid',
                            'rows': 3
                        }
                    )
                ], className="four columns"),
                html.Div([
                    'cytoscape-circle',
                    cyto.Cytoscape(
                        id='cytoscape-circle',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'circle',
                            'radius': 250,
                            'startAngle': math.pi * 1 / 6,
                            'sweep': math.pi * 2 / 3
                        }
                    )
                ], className="four columns"),
                html.Div([
                    'cytoscape-breadthfirst1',
                    cyto.Cytoscape(
                        id='cytoscape-breadthfirst1',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'breadthfirst',
                            'roots': '[id = "nyc"]' # специальный синтаксис для указания ID узлов 
                        }
                    )
                ], className="four columns"),
            ],
            className='Row'
        ),
        html.Div(
            [
                html.Div([
                    'cytoscape-breadthfirst2',
                    cyto.Cytoscape(
                        id='cytoscape-breadthfirst2',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'breadthfirst',
                            'roots': '#van, #mtl' # специальный синтаксис для указания ID узлов 
                        }
                    )
                ], className="four columns"),
                html.Div([
                    'cytoscape-cose',
                    cyto.Cytoscape(
                        id='cytoscape-cose',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'cose'
                        }
                    )
                ], className="four columns"),
            ],
            className="Row"
        )
    ]
)

if __name__ == '__main__':
    app.run_server(debug=True)


Overwriting 06_dash_cytoscape/layout.py


![Cyto Layout](06_dash_cytoscape/cyto_layout.png)

## Стилизация в Cytoscape

Стили описываются в виде словарей, содержащих 2 ключа: `selector` для идентификации элемента, к которому будет применен стиль, и `style` - собственно, стиль (высота, ширина, цвет, ...).

Селекторы бывают 2 типов:
* селекторы группы: node или edge
* селекторы класса

Специальные возможности в селекторах классов ([полный список](https://js.cytoscape.org/#selectors/data))
```
"selector": "[weight <= 3]" # значение атрибута weight <=3
"selector": "[firstname  *= 'ert']" # в значении атрибута firstname есть подстрока 'ert'
"selector": "[firstname  !*= 'ert']" # в значении атрибута firstname нет подстроки 'ert'
"selector": "[firstname  ^= 'Alb']" # значение атрибута firstname начинается с подстроки 'Alb'
"selector": "[firstname  @^= 'Alb']" # значение атрибута firstname начинается с подстроки 'Alb' (без учета регистра)
"selector": "#AD, #DA" # ребро от AD к DA
```

Стили ребер:
* line-: 
    * color
    * style
    * cap 
    * opacity
    * fill (solid, gradient, radial-gradient)
    * dash-pattern (модификатор для пунктирных линий, длина черточек и пропусков: [6, 3])
    * dash-offset   
* curve-style:
    - haystack: встроенный быстрый вариант в виде прямой без циклов и составных узлов
    - straight - прямые со стрелками
    - straight-triangle - прямые с треугольными стрелками
    - bezier - закругленные 
    - segments - ломаные
    - taxi
* [mid-]source(target)-arrow-
    - color
    - shape
* arrow-scale

На узлы можно добавлять изображения при помощи стилей:
```
'style': {
            'width': 90,
            'height': 80,
            'background-fit': 'cover',
            'background-image': 'data(url)'
        }
```


In [81]:
%%file 06_dash_cytoscape/style.py
from dash import Dash, html, dcc
import dash_cytoscape as cyto
import networkx as nx
import math

def city_graph():
    nodes = [
        {
            'data': {'id': short, 'label': label, 'pop': pop},
            'position': {'x': 20 * lat, 'y': -20 * long}
        }
        for short, label, long, lat, pop in (
            ('la', 'Los Angeles', 34.03, -118.25, 1000),
            ('nyc', 'New York', 40.71, -74, 2000),
            ('to', 'Toronto', 43.65, -79.38, 1000),
            ('mtl', 'Montreal', 45.50, -73.57, 3000),
            ('van', 'Vancouver', 49.28, -123.12, 6000),
            ('chi', 'Chicago', 41.88, -87.63, 3000),
            ('bos', 'Boston', 42.36, -71.06, 5000),
            ('hou', 'Houston', 29.76, -95.37, 1000)
        )
    ]

    edges = [
        {'data': {'id':f'{source}2{target}', 'source': source, 'target': target}}
        for source, target in (
            ('van', 'la'),
            ('la', 'chi'),
            ('hou', 'chi'),
            ('to', 'mtl'),
            ('mtl', 'bos'),
            ('nyc', 'bos'),
            ('to', 'hou'),
            ('to', 'nyc'),
            ('la', 'nyc'),
            ('bos', 'nyc')
        )
    ]
    elements = nodes + edges
    return elements


app = Dash(
    __name__,
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
)
elements = city_graph()

app.layout = html.Div([
        html.Div(
            [
                html.Div([
                    'Красные узлы содержат пробел в названии',
                    cyto.Cytoscape(
                        id='cytoscape-grid',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'grid',
                            'rows': 3
                        },
                        stylesheet=[
                            {
                                "selector": "[label *= ' ']",
                                "style": {
                                    "background-color": "#FF4136",
                                }
                            }
                        ]
                    )
                ], className="four columns"),
                html.Div([
                    'Зеленые узлы имеют степень больше 2 + кривые ребра для параллельных связей',
                    cyto.Cytoscape(
                        id='cytoscape-circle',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'circle',
                            'radius': 250,
                            'startAngle': math.pi * 1 / 6,
                            'sweep': math.pi * 2 / 3
                        },
                        stylesheet=[
                            {
                                "selector": "node",
                                'style': {
                                    'label': 'data(label)'
                                }
                            },
                            {
                                "selector": "[[degree > 2]]", # двойные скобки для метаинформации
                                "style": {
                                    "background-color": "green"
                                }
                            },
                            {
                                "selector": '#nyc2bos, #bos2nyc', # способ выбрать ребра
                                "style": {
                                    'curve-style': 'bezier',
                                    'label': 'bezier',
                                }
                            }
                        ]
                    )
                ], className="four columns"),
                html.Div([
                    'Треугольные узлы имеют большое население; Хьюстон раскрашен в зеленый',
                    cyto.Cytoscape(
                        id='cytoscape-breadthfirst1',
                        elements=elements,
                        style={'width': '100%', 'height': '350px'},
                        layout={
                            'name': 'breadthfirst',
                            'roots': '[id = "nyc"]' # специальный синтаксис для указания ID узлов 
                        },
                        stylesheet=[
                            {
                                'selector': 'node',
                                'style': {
                                    'label': 'data(label)'
                                }
                            },
                            {
                                "selector": "[pop >= 3000]",
                                "style": {
                                    "shape": "triangle"
                                }
                            },
                            {
                                "selector": "[label = 'Houston']",
                                "style": {
                                    "shape": "rectangle"
                                }
                            },
                            {
                                "selector": "#nyc2bos, #bos2nyc",
                                "style": {
                                    "curve-style": "bezier"
                                }
                            },
                            {
                                "selector": "#nyc2la, #la2nyc",
                                "style": {
                                    "curve-style": "segments"
                                }
                            },
                            {
                                "selector": "#to2nyc",
                                "style": {
                                    'target-arrow-color': 'blue',
                                    'target-arrow-shape': 'vee', # некоторые типы ребер не работают со стрелочками
                                    "curve-style": "bezier",
                                    'line-color': 'red',
                                }
                            },
                        ]
                    )
                ], className="four columns"),
            ],
            className='Row'
        ),
    ]
)

if __name__ == '__main__':
    app.run_server(debug=True)


Overwriting 06_dash_cytoscape/style.py


![Cyto Styling](06_dash_cytoscape/cyto_style.png)

## Функции обратного вызова

Обратные вызовы Dash позволяют обновлять график Cytoscape с помощью других компонентов, таких как выпадающие списки, кнопки и ползунки

In [105]:
%%file 06_dash_cytoscape/callbacks.py
from dash import Dash, html, dcc, Input, Output, State
import dash_cytoscape as cyto
import networkx as nx
import math

def city_graph():
    nodes = [
        {
            'data': {'id': short, 'label': label, 'pop': pop},
            'position': {'x': 20 * lat, 'y': -20 * long}
        }
        for short, label, long, lat, pop in (
            ('la', 'Los Angeles', 34.03, -118.25, 1000),
            ('nyc', 'New York', 40.71, -74, 2000),
            ('to', 'Toronto', 43.65, -79.38, 1000),
            ('mtl', 'Montreal', 45.50, -73.57, 3000),
            ('van', 'Vancouver', 49.28, -123.12, 6000),
            ('chi', 'Chicago', 41.88, -87.63, 3000),
            ('bos', 'Boston', 42.36, -71.06, 5000),
            ('hou', 'Houston', 29.76, -95.37, 1000)
        )
    ]

    edges = [
        {'data': {'id':f'{source}2{target}', 'source': source, 'target': target}}
        for source, target in (
            ('van', 'la'),
            ('la', 'chi'),
            ('hou', 'chi'),
            ('to', 'mtl'),
            ('mtl', 'bos'),
            ('nyc', 'bos'),
            ('to', 'hou'),
            ('to', 'nyc'),
            ('la', 'nyc'),
            ('bos', 'nyc')
        )
    ]
    return nodes, edges


app = Dash(
    __name__,
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
)
nodes, edges = city_graph()
elements = nodes + edges

default_stylesheet = [
    {
        'selector': 'node',
        'style': {
            'background-color': '#BFD7B5',
            'label': 'data(label)'
        }
    },
    {
        'selector': 'edge',
        'style': {
            'line-color': '#A3C4BC'
        }
    }
]

app.layout = html.Div(
    [
        cyto.Cytoscape(
            id="cyto",
            elements=elements,
            stylesheet=default_stylesheet,
            style={
                "width": "100%",
                "height": "400px"
            },
            layout={
                "name": "cose"
            }
        ),
        html.Div(
            [
                html.Div(
                    [
                        'Цвет связей',
                        html.Br(),
                        dcc.Input(id="line-color")
                    ],
                    className='two columns'
                ),
                html.Div(
                    [
                        'Цвет узлов',
                        html.Br(),
                        dcc.Input(id="node-color")
                    ],
                    className='two columns'
                ),
                html.Div(
                    [
                        'Стиль расположения узлов',
                        dcc.Dropdown(
                            options=[
                                "cose",
                                "random",
                                "circle",
                                "concentric",
                                "grid"
                            ],
                            value="cose",
                            id="layout-dropdown"
                        )
                    ],
                    className='two columns'
                ),
            ],
            style={
                "display": "flex"
            }
        ),
        html.Br(),
        html.Div(
            [
                html.Div(
                    [
                        html.Button("Добавить узел", id="add_button", n_clicks_timestamp=0)
                    ],
                    className='two columns'
                ),
                html.Div(
                    [
                        html.Button("Удалить узел", id="del_button", n_clicks_timestamp=0)
                    ],
                    className='two columns'
                ),
            ],
            style={
                "display": "flex"
            }
        )
    ]
)

@app.callback(
    Output("cyto", "layout"),
    Input("layout-dropdown", "value")
)
def _(layout):
    return {
        "name": layout,
        'animate': True
    }

@app.callback(
    Output("cyto", "stylesheet"),
    Input("node-color", "value"),
    Input("line-color", "value"),
)
def _(node_color, line_color):
    # если цвет не распознан, функция отработает
    # но ничего не произвойдет
    if line_color is None:
        line_color = ''

    if node_color is None:
        node_color = ''

    new_styles = [
        {
            'selector': 'node',
            'style': {
                'background-color': node_color
            }
        },
        {
            'selector': 'edge',
            'style': {
                'line-color': line_color
            }
        }
    ]
    # не добавляем новые стили непосредственно к стилю по умолчанию, 
    # а вместо этого объединяем default_stylesheet с new_styles. 
    # Это связано с тем, что любое изменение default_stylesheet будет постоянным, 
    # что не очень хорошо, если вы размещаете свое приложение 
    # для многих пользователей 
    # (поскольку default_stylesheet является общим для всех пользовательских сеансов).
    return default_stylesheet + new_styles

@app.callback(
    Output('cyto', 'elements'),
    Input('add_button', 'n_clicks_timestamp'),
    Input('del_button', 'n_clicks_timestamp'),
    State('cyto', 'elements')
)
def _(click_add, click_del, elements):
    def get_current_and_deleted_nodes(elements):
        current_nodes, deleted_nodes = [], []
        current_node_ids = set()
        for elem in elements:
            # у узлов нет ключа source
            if 'source' not in elem['data']:
                current_nodes.append(elem)
                current_node_ids.add(elem['data']['id'])

        # nodes - глобальный список узлов
        for node in nodes:
            if node['data']['id'] not in current_node_ids:
                deleted_nodes.append(node)
        return current_nodes, deleted_nodes

    def get_edges_for_current_nodes(current_nodes):
        # edges - глобальный список ребер
        current_node_ids = {node['data']['id'] for node in current_nodes}
        current_edges = []
        for edge in edges:
            if edge['data']['source'] in current_node_ids and edge['data']['target'] in current_node_ids:
                current_edges.append(edge)
        return current_edges

    add_clicked = int(click_add) > int(click_del) # последний клик - по кнопке добавить
    del_clicked = int(click_add) < int(click_del) # последний клик - по кнопке удалить
    current_nodes, deleted_nodes = get_current_and_deleted_nodes(elements)
    if add_clicked and deleted_nodes:
        current_nodes.append(deleted_nodes.pop())
        return current_nodes + get_edges_for_current_nodes(current_nodes)
    elif del_clicked and current_nodes:
        current_nodes.pop()
        return current_nodes + get_edges_for_current_nodes(current_nodes)
    return elements


if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting 06_dash_cytoscape/callbacks.py


![Cyto Callbacks](06_dash_cytoscape/cyto_callbacks.png)

## Взаимодействие с пользователем

Dash позволяет модифицировать стили, макет и элементы графа Cytoscape при помощи других визуальных элементов. Кроме этого, в качестве входов для обратных вызовов можно использовать и свойства самого Cytoscape. Такие функции обратного вызова называются функциями обратного вызова для событий.

Ниже перечислены несколько свойств такого рода:
- tapNode: возвращает полное описание объекта узла, когда пользователь щелкает или нажимает на узел;
- tapNodeData: словарь данных узла, возвращаемый при нажатии или щелчке по нему;
- tapEdge: возвращает полное описание объекта узла, когда пользователь щелкает или нажимает на узел;   
- tapEdgeData : словарь данных ребра, возвращаемый при нажатии или щелчке по нему;
- mouseoverEdgeData: возвращает только словарь данных связи, на которую навелся пользователь
- mouseoverNodeData: возвращает только словарь данных узла, на которую навелся пользователь
- selectedNodeData: список словарей данных всех выбранных узлов (например, выбранных при помощи shift+click);
- selectedEdgeData: список словарей данных всех выбранных ребер (например, выбранных при помощи shift+click);


In [21]:
%%file 06_dash_cytoscape/event_callbacks.py
from dash import Dash, html, dcc, Input, Output, State
import dash_cytoscape as cyto
import networkx as nx
import json


def city_graph():
    nodes = [
        {
            'data': {'id': short, 'label': label, 'pop': pop},
            'position': {'x': 20 * lat, 'y': -20 * long}
        }
        for short, label, long, lat, pop in (
            ('la', 'Los Angeles', 34.03, -118.25, 1000),
            ('nyc', 'New York', 40.71, -74, 2000),
            ('to', 'Toronto', 43.65, -79.38, 1000),
            ('mtl', 'Montreal', 45.50, -73.57, 3000),
            ('van', 'Vancouver', 49.28, -123.12, 6000),
            ('chi', 'Chicago', 41.88, -87.63, 3000),
            ('bos', 'Boston', 42.36, -71.06, 5000),
            ('hou', 'Houston', 29.76, -95.37, 1000)
        )
    ]

    edges = [
        {'data': {'id':f'{source}2{target}', 'source': source, 'target': target}}
        for source, target in (
            ('van', 'la'),
            ('la', 'chi'),
            ('hou', 'chi'),
            ('to', 'mtl'),
            ('mtl', 'bos'),
            ('nyc', 'bos'),
            ('to', 'hou'),
            ('to', 'nyc'),
            ('la', 'nyc'),
            ('bos', 'nyc')
        )
    ]
    return nodes, edges


app = Dash(
    __name__,
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
)
nodes, edges = city_graph()
elements = nodes + edges

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

default_stylesheet = [
    {
        'selector': 'node',
        'style': {
            'background-color': '#BFD7B5',
            'label': 'data(label)'
        }
    },
    {
        'selector': 'edge',
        'style': {
            'line-color': '#A3C4BC'
        }
    }
]

app.layout = html.Div(
    [
        cyto.Cytoscape(
            id="cyto",
            elements=elements,
            stylesheet=default_stylesheet,
            style={
                "width": "100%",
                "height": "400px"
            },
            layout={
                "name": "preset"
            }
        ),
        html.Div(
            [
                html.Div(
                    [
                        "Данные об узле при клике",
                        html.Pre(id='tap-node-data-pre', style=styles['pre']),
                    ],
                    className="two columns"
                ),
                html.Div(
                    [
                        "Данные об ребре при клике",
                        html.Pre(id='tap-edge-data-pre', style=styles['pre']),
                    ],
                    className="two columns"
                ),
                html.Div(
                    [
                        "Данные об узле при наведении",
                        html.Pre(id='hover-node-data-pre', style=styles['pre']),
                    ],
                    className="two columns"
                ),
                html.Div(
                    [
                        "Данные об ребре при наведении",
                        html.Pre(id='hover-edge-data-pre', style=styles['pre']),
                    ],
                    className="two columns"
                ),
                html.Div(
                    [
                        "Выбранные узлы",
                        html.Pre(id='selected-node-data-pre', style=styles['pre']),
                    ],
                    className="two columns"
                ),
            ],
            className="Row"
        )
    ]
)

@app.callback(
    Output('tap-node-data-pre', 'children'),
    Input('cyto', 'tapNodeData')
)
def _(data):
    return json.dumps(data, indent=2)

@app.callback(
    Output('tap-edge-data-pre', 'children'),
    Input('cyto', 'tapEdgeData')
)
def _(data):
    return json.dumps(data, indent=2)

@app.callback(
    Output('hover-node-data-pre', 'children'),
    Input('cyto', 'mouseoverNodeData')
)
def _(data):
    return json.dumps(data, indent=2)

@app.callback(
    Output('hover-edge-data-pre', 'children'),
    Input('cyto', 'mouseoverEdgeData')
)
def _(data):
    return json.dumps(data, indent=2)

@app.callback(
    Output('selected-node-data-pre', 'children'),
    Input('cyto', 'selectedNodeData')
)
def _(data):
    return json.dumps(data, indent=2)

@app.callback(
    Output("cyto", "stylesheet"),
    Input("cyto", "tapNode")
)
def _(tap_data):
    if tap_data is None:
        return default_stylesheet

    node_id = tap_data['data']['id']
    additional_styles = [
        {
            "selector": f"#{node_id}",
            "style": {
                "background-color": "green"
            }
        }
    ]
    for edge in tap_data['edgesData']:
        for node_side in ["source", "target"]:
            if edge[node_side] != node_id:
                additional_styles.append(
                    {
                        "selector": f"#{edge[node_side]}",
                        "style": {
                            "background-color": "pink"
                        }
                    }
                )
    return default_stylesheet + additional_styles

if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting 06_dash_cytoscape/event_callbacks.py


![Cyto Event Callbacks](06_dash_cytoscape/cyto_event_callbacks.png)

## Работа с большим графом

https://blog.js.cytoscape.org/2020/05/11/layouts/

https://js.cytoscape.org/#layouts

https://stackoverflow.com/questions/61370691/change-individual-node-size-using-dash-cytoscape

In [92]:
%%file 06_dash_cytoscape/webgraph.py
from dash import Dash, html, dcc, Input, Output, State
import dash_cytoscape as cyto
import networkx as nx
import json

import pandas as pd
import networkx as nx

def fb_graph():
    nodes_df = pd.read_csv('06_dash_cytoscape/musae_facebook_target.csv')
    edges_df = pd.read_csv('06_dash_cytoscape/musae_facebook_edges.csv')
    G = nx.Graph()
    G.add_edges_from(edges_df.values.tolist())
    layout = nx.circular_layout(G)

    edges = []
    for u, v in edges_df.values:
        edges.append({'data': {'source': u, 'target': v}})
    nodes = []
    for node in nodes_df.to_dict(orient='records'):
        id_ = node['id']
        nodes.append({
            'data': node, 
            # 'position': {
            #     'x': layout[id_][0], 
            #     'y': layout[id_][1]
            # },
            # 'locked': True,
            # 'selectable': False,
            # 'draggable': False
        })    

    return nodes, edges

def city_graph():
    nodes = [
        {
            'data': {'id': short, 'label': label, 'pop': pop},
            'position': {'x': 20 * lat, 'y': -20 * long}
        }
        for short, label, long, lat, pop in (
            ('la', 'Los Angeles', 34.03, -118.25, 1000),
            ('nyc', 'New York', 40.71, -74, 2000),
            ('to', 'Toronto', 43.65, -79.38, 1000),
            ('mtl', 'Montreal', 45.50, -73.57, 3000),
            ('van', 'Vancouver', 49.28, -123.12, 6000),
            ('chi', 'Chicago', 41.88, -87.63, 3000),
            ('bos', 'Boston', 42.36, -71.06, 5000),
            ('hou', 'Houston', 29.76, -95.37, 1000)
        )
    ]

    edges = [
        {'data': {'id':f'{source}2{target}', 'source': source, 'target': target}}
        for source, target in (
            ('van', 'la'),
            ('la', 'chi'),
            ('hou', 'chi'),
            ('to', 'mtl'),
            ('mtl', 'bos'),
            ('nyc', 'bos'),
            ('to', 'hou'),
            ('to', 'nyc'),
            ('la', 'nyc'),
            ('bos', 'nyc')
        )
    ]
    return nodes, edges

def web_graph():
    node_ids = set()
    nodes = []
    edges = []
    with open("06_dash_cytoscape/web-webbase-2001.mtx") as fp:
        fp.readline()
        fp.readline()
        for line in fp:
            source, target = tuple(map(int, line.split()))
            node_ids.update((source, target))
            edges.append({'data': {'source': source, 'target': target}})
    for node_id in node_ids:
        nodes.append({'data': {'id': node_id}})
    return nodes, edges

app = Dash(
    __name__,
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
)
nodes, edges = fb_graph()
elements = nodes + edges

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

default_stylesheet = [
    {
        'selector': 'node',
        'style': {
            'background-color': '#BFD7B5',
            'width': "3px",
            'height': "3px",
            "border-color": "black",
            "border-width": 1,
        },

    },
    {
        'selector': 'edge',
        'style': {
            'line-color': 'gray',
            'width': "1px",
            'height': "1px",
        }
    }
]

app.layout = html.Div(
    [
        html.P(
            f"Граф facebook: {len(nodes)} узлов, {len(edges)} связей", 
            style={"text-align": "center"}
        ),
        cyto.Cytoscape(
            id="cyto",
            elements=elements,
            stylesheet=default_stylesheet,
            style={
                "width": "100%",
                "height": "1000px"
            },
            layout={
                "name": "concentric",
            },
            responsive=False
        )
    ]
)


if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting 06_dash_cytoscape/webgraph.py
